# Calibration of ADCs

In [1]:
import numpy as np
import pyfftw

import ROOT
from ROOT import gROOT
from ROOT import gStyle
%jsroot on

from array import array
#--------------------------------------------

gROOT.LoadMacro("../style/CMS/tdrstyle.C")
gROOT.ProcessLine("setTDRStyle();")
gROOT.LoadMacro("../style/CMS/CMS_lumi.C")

gStyle.SetOptFit(0)

Welcome to JupyROOT 6.24/00


In [2]:
class Signal(object):
    class header(object):
        ev = 0
        ch = 0
    class peak(object):
        index = 0
        value = 0
    width = 0
    integral = 0
    data = []

In [3]:
# Remove Pedestal
def subPedestal(data, pos):
    fft = pyfftw.interfaces.numpy_fft.fft(data)
    fft[0] = 0 
    ifft = pyfftw.interfaces.numpy_fft.ifft(fft).real
    idata = array('f')
    for i in range(len(ifft)):
        idata.append(pos*round(ifft[i]));
    return idata

In [4]:
#get signals from array
from scipy.signal import find_peaks,peak_widths
def findSignals(data, ev = 0, ch = 0, thr=50, pad=30):
    output = []
    peaks, _ = find_peaks(idata, height=thr)
    for p in range(len(peaks)):
        sig = Signal()
        sig.header.ev = ev 
        sig.header.ch = ch
        sig.peak.index = peaks[p]
        sig.peak.value = data[peaks[p]]
        sig.width = peak_widths(data, peaks, rel_height=0.2)[0][p]
        sig.integral = sum(data[peaks[p] - pad: peaks[p] + pad])
        sig.data = data[peaks[p] - pad: peaks[p] + pad]
        output.append(sig)
    return output

In [5]:
ROOT.gInterpreter.AddIncludePath('../../include/')
ROOT.gInterpreter.Declare('#include "RoEvent.h"')
ROOT.gSystem.Load('../../libs/libRoEvent.so')
#get data from RoEvent class (to generate it use dq script)
f = ROOT.TFile("../../tmp/ro_r601.root")
myTree = f.Get("data")
#myTree.Print()
myTree.Show(560)

======> EVENT:560
 events          = (RoEvent*)0x7f3e3af25780
 fUniqueID       = 0
 fBits           = 33554432
 fEvtHdr.fUniqueID = 0
 fEvtHdr.fBits   = 33554432
 fEvtHdr.fEvtNum = 561
 fEvtHdr.fValid  = 1
 fEvtHdr.fGood   = 0
 fEvtHdr.fFlags[8] = 0 , 31024 , 49827 , 32767 , 0 , 1 , 0 , 0 

 fEvtHdr.fRunNum = 601
 fEvtHdr.fRunFlags = 32767
 fEvtHdr.fTrigType = 0
 fEvtHdr.fTrigNum = 561
 fEvtHdr.fTvNsec = 275592696
 fNumPayloads    = 64
 fPayloads       = 64
 fPayloads.fUniqueID = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
 fPayloads.fBits = 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648, 50331648
 fPayloads.fCh   = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19
 fPayloads.fMap.fUniqueID = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
 fPayloads.fMap.fBits = 50331648, 50331648, 50331648, 50331648,

Warning in <TFile::Init>: no StreamerInfo found in /home/const/projects/CRO/libs/libRoEvent_rdict.pcm therefore preventing schema evolution when reading this file. The file was produced with version 6.22/06 of ROOT.
Error in <TUnixSystem::Load>: version mismatch, /home/const/projects/CRO/libs/libRoEvent.so = 62206, ROOT = 62400


In [6]:
# LOOP over channal
thr = 70; pad = 30;  pol = -1;

#signals = [[] for i in range(64)]
p_root = ROOT.TH2F("h","",1005,0,1004,64,0,63);      
for entry in myTree:
    
    events = entry.events
    numCh = events.getNumPayloads()
    ev = events.getHeader().getEvtNum()
    print("Ev {} is processing... ".format(ev), end='\r')
    for n in range(numCh):
        ch = events.getPayloads().AddrAt(n).getCh()
        cro = events.getPayloads().AddrAt(n).getCRO()
        numSa = events.getPayloads().AddrAt(n).getNsaCRO()
        data = array('i')
        for ns in range(numSa): 
            data.append(int(cro[ns]));
        idata = subPedestal(data, pol)
        ev_signals = findSignals(idata, ev, ch, thr, pad)
        for i in range(len(ev_signals)):
            p_root.Fill(ev_signals[i].header.ev, ev_signals[i].header.ch)
        
        #signals[n].extend(ev_signals)

In [8]:
p_root_s = p_root.Clone()

pathToSave = "/mnt/d/tmp/"
#---
iPeriod = "0"; iPos="0"
#---
gROOT.ProcessLine("cmsText = \"Cryo Lyon\";")
gROOT.ProcessLine("writeExtraText = true;") # if extra text
#gROOT.ProcessLine("writeExtraText = false;") #remove Preliminary
gROOT.ProcessLine("extraText  = \"\";") # default extra text is "Preliminary"
gROOT.ProcessLine("lumi_sqrtS = \" #bf{negative (thr70ADCu)} | short:0-31ch  | MUX chs scan | run:601 12-05-21 \";") 


p_root_s.SetMarkerColor(1); p_root_s.SetMarkerStyle(20); p_root_s.SetMarkerSize(0.2)
p_root_s.SetLineColor(1); p_root_s.GetXaxis().SetTitle(""); p_root_s.GetYaxis().SetTitle("")

cName = "r601_MUXtest_neg"; cTitle = "c"; cX = 800; cY = 600
c = ROOT.TCanvas(cName, cTitle, cX, cY)
#c.SetGrid()
#---

p1 = ROOT.TPad("p1", "", 0, 0, 1, 1);
p1.SetGrid();

# Draw the 1st TMultiGraph
p1.SetBottomMargin(0.2)
p1.Draw();
p1.cd();

p_root_s.Draw(); 

p_root_s.GetXaxis().SetLabelSize(0);
p_root_s.GetXaxis().SetTickLength(0);
p_root_s.GetYaxis().SetLabelSize(0);
p_root_s.GetYaxis().SetTickLength(0); 

xmin = p_root.GetXaxis().GetXmin();                       
xmax = p_root.GetXaxis().GetXmax();
ymin = p_root.GetYaxis().GetXmin();
ymax = p_root.GetYaxis().GetXmax();

ymin = 0
ymax = 31
p_root_s.GetYaxis().SetRangeUser(0, 31);

dx = (xmax - xmin) / 0.8;
dy = (ymax - ymin) / 0.8;
yaxis1 = ROOT.TGaxis(xmin, ymin, xmin, ymax, ymin + 0.5, ymax + 0.5, 510,"L");
yaxis1.SetName("yaxis1");
yaxis1.SetTitle("chid")
yaxis1.SetLabelSize(0.03);
yaxis1.SetLabelOffset(0.03);
yaxis1.SetTitleSize(0.03);
yaxis1.SetTitleOffset(1.2)
yaxis1.Draw();

xaxis1 = ROOT.TGaxis(xmin, ymin, xmax, ymin, xmin, xmax, 510, "L");
xaxis1.SetName("xaxis1");
xaxis1.SetTitle("evno  ")
xaxis1.SetTitleOffset(1)
xaxis1.SetLabelSize(0.03);
xaxis1.SetLabelOffset(0.01);
xaxis1.SetTitleSize(0.03);
xaxis1.Draw(); 


xaxis2 = ROOT.TGaxis(xmin + 15 + 16, ymin - 3.5, xmax - 15, ymin - 3.5, 0, 31, 510, "L");
xaxis2.SetName("xaxis2");
xaxis2.SetTitle("MUXch ")
xaxis2.SetTitleOffset(1.15)
xaxis2.SetLabelSize(0.03);
xaxis2.SetTitleSize(0.03);
xaxis2.Draw();    

cms_lumi_str = "CMS_lumi("+cName+", "+iPeriod+", "+iPos+");"; gROOT.ProcessLine(cms_lumi_str)

c.SaveAs("/mnt/d/" + cName + ".svg");
c.SaveAs("/mnt/d/" + cName + ".pdf");
p_root.SaveAs("/mnt/d/" + cName + ".root");
c.Draw()


#--- 

 #bf{negative (thr70ADCu)} | short:0-31ch  | MUX chs scan | run:601 12-05-21 


Warning in <TCanvas::Constructor>: Deleting canvas with same name: r601_MUXtest_neg
Info in <TCanvas::Print>: SVG file /mnt/d/r601_MUXtest_neg.svg has been created
Info in <TCanvas::Print>: pdf file /mnt/d/r601_MUXtest_neg.pdf has been created
Info in <TH2F::SaveAs>: ROOT file /mnt/d/r601_MUXtest_neg.root has been created


In [34]:
inputFile0 = ROOT.TFile("/mnt/d/r601_MUXtest_pos.root")
#obj0 = inputFile0.gr_0_middle_CBefficiency; obj0.SetTitle("")
obj0 = inputFile0.h; obj0.SetTitle("")
cName = "r601_MUXtest_pos"; cTitle = "c"; cX = 800; cY = 600
c = ROOT.TCanvas(cName, cTitle, cX, cY)
obj0.Draw(); 
c.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: r601_MUXtest_pos
